In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np

import osu.dataset as dataset


In [ ]:
# reload from saved

xs = np.load(f'.datasets/xs_642_08-13_01-26-33.npy')
ys = np.load(f'.datasets/ys_642_08-13_01-26-33.npy')

# xs = np.load(f'.datasets/xs_142_07-21_21-50-55.npy')
# ys = np.load(f'.datasets/ys_142_07-21_21-50-55.npy')

In [ ]:
xs.shape

In [ ]:
import torch

BATCH_SIZE = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

In [ ]:
# from models.rnn import OsuReplayRNN
# 
# rnn = OsuReplayRNN(batch_size=BATCH_SIZE, noise_std=0.045)
# 
# # vae = OsuReplayVAE.load(".trained/vae_353rcl_9ft.pt")
# 
# rnn.load_data(xs, ys)

In [ ]:
from models.vae import OsuReplayVAE
from models.annealer import Annealer

annealer = Annealer(
    total_steps=10,
    range=(0, 0.3),
    cyclical=True,
    stay_max_steps=5
)

# vae = OsuReplayVAE(batch_size=BATCH_SIZE, annealer=annealer, noise_std=0.045)

# vae = OsuReplayVAE.load(".trained/vae_353rcl_9ft.pt")
vae = OsuReplayVAE.load(".trained/replayvae_most_recent.pt", annealer=annealer, noise_std=0.03)

vae.load_data(xs, ys)

In [ ]:
from models.keys import OsuKeyModel

keys = OsuKeyModel(batch_size=BATCH_SIZE)
keys.load_data(xs, ys)

In [ ]:
model = vae

EPOCHS = 3

for i in range(25):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [ ]:
for i in range(50):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [ ]:
for i in range(150):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [ ]:
for i in range(150):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [ ]:
del vae

In [ ]:
annealer = Annealer(
    total_steps=10,
    range=(0, 0.25),
    cyclical=True,
    stay_max_steps=5
)

vae = OsuReplayVAE(batch_size=BATCH_SIZE, annealer=annealer, noise_std=0.045)

# vae = OsuReplayVAE.load(".trained/vae_353rcl_9ft.pt")
# vae = OsuReplayVAE.load(".trained/vae_most_recent.pt")

vae.load_data(xs, ys)

model = vae

In [ ]:
for i in range(175):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [ ]:
for i in range(300):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [ ]:
# test training results
from osu.rulesets.mods import Mods
import osu.rulesets.beatmap as bm
import osu.dataset as dataset

test_name = 'zuttosummer'
test_mods = Mods.DOUBLE_TIME
test_map_path = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'

test_map = bm.load(test_map_path)
test_map.apply_mods(test_mods)

data = dataset.input_data(test_map)
data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
data = torch.FloatTensor(data)

In [ ]:
replay_data = vae.generate(data)
    
replay_data = np.concatenate(replay_data)
replay_data = np.pad(replay_data, ((0, 0), (0, 2)), mode='constant', constant_values=0)
if not os.path.exists('.generated'):
    os.makedirs('.generated')
    
print(f"Generated replay data shape: {replay_data.shape}")


In [ ]:
import osu.preview.preview as preview

preview.preview_replay_raw(replay_data, test_map_path, test_mods, test_song)